# Chapter 10: Sparse matrices and graphs

Robert Johansson

Source code listings for [Numerical Python - A Practical Techniques Approach for Industry](http://www.apress.com/9781484205549) (ISBN 978-1-484205-54-9).

The source code listings can be downloaded from http://www.apress.com/9781484205549

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
# mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.sans-serif'] = 'stix'

In [ ]:
import scipy.sparse as sp

In [ ]:
import scipy.sparse.linalg

In [ ]:
import numpy as np

In [ ]:
import scipy.linalg as la

In [ ]:
import networkx as nx

## Coordinate list format

In [ ]:
values = [1, 2, 3, 4]

In [ ]:
rows = [0, 1, 2, 3]

In [ ]:
cols = [1, 3, 2, 0]

In [ ]:
A = sp.coo_matrix((values, (rows, cols)), shape=[4, 4])

In [ ]:
A.todense()

In [ ]:
A

In [ ]:
A.shape, A.size, A.dtype, A.ndim

In [ ]:
A.nnz, A.data

In [ ]:
A.row

In [ ]:
A.col

In [ ]:
A.tocsr()

In [ ]:
A.toarray()

In [ ]:
A.todense()

Not all sparse matrix formats supports indexing:

In [ ]:
A[1, 2]

In [ ]:
A.tobsr()[1, 2]

But some do:

In [ ]:
A.tocsr()[1, 2]

In [ ]:
A.tolil()[1:3, 3]

## CSR

In [ ]:
A = np.array([[1, 2, 0, 0], [0, 3, 4, 0], [0, 0, 5, 6], [7, 0, 8, 9]]); A

In [ ]:
A = sp.csr_matrix(A)

In [ ]:
A.data

In [ ]:
A.indices

In [ ]:
A.indptr

In [ ]:
i = 2

In [ ]:
A.indptr[i], A.indptr[i+1]-1

In [ ]:
A.indices[A.indptr[i]:A.indptr[i+1]]

In [ ]:
A.data[A.indptr[i]:A.indptr[i+1]]

## Functions for constructing sparse matrices

In [ ]:
N = 10

In [ ]:
A = -2 * sp.eye(N) + sp.eye(N, k=1) + sp.eye(N, k=-1)

In [ ]:
A

In [ ]:
A.todense()

In [ ]:
fig, ax = plt.subplots()
ax.spy(A)
fig.savefig("ch10-sparse-matrix-1.pdf");

In [ ]:
A = sp.diags([1,-2,1], [1,0,-1], shape=[N, N], format='csc')

In [ ]:
A

In [ ]:
fig, ax = plt.subplots()
ax.spy(A);

In [ ]:
B = sp.diags([1, 1], [-1, 1], shape=[3,3])

In [ ]:
B

In [ ]:
C = sp.kron(A, B, format='csr')
C

In [ ]:
fig, (ax_A, ax_B, ax_C) = plt.subplots(1, 3, figsize=(12, 4))
ax_A.spy(A)
ax_B.spy(B)
ax_C.spy(C)
fig.savefig("ch10-sparse-matrix-2.pdf");

## Sparse linear algebra

In [ ]:
N = 10

In [ ]:
A = sp.diags([1, -2, 1], [1, 0, -1], shape=[N, N], format='csc')

In [ ]:
b = -np.ones(N)

In [ ]:
x = sp.linalg.spsolve(A, b)

In [ ]:
x

In [ ]:
np.linalg.solve(A.todense(), b)

In [ ]:
lu = sp.linalg.splu(A)

In [ ]:
lu.L

In [ ]:
lu.perm_r

In [ ]:
lu.U

In [ ]:
def sp_permute(A, perm_r, perm_c):
    """ permute rows and columns of A """
    M, N = A.shape
    # row permumation matrix
    Pr = sp.coo_matrix((np.ones(M), (perm_r, np.arange(N)))).tocsr()
    # column permutation matrix
    Pc = sp.coo_matrix((np.ones(M), (np.arange(M), perm_c))).tocsr()
    return Pr.T * A * Pc.T

In [ ]:
sp_permute(lu.L * lu.U, lu.perm_r, lu.perm_c) - A

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))
ax1.spy(lu.L)
ax2.spy(lu.U)
ax3.spy(A)

In [ ]:
x = lu.solve(b)

In [ ]:
x

In [ ]:
# use_umfpack=True is only effective if scikit-umfpack is installed
# (in which case UMFPACK is the default solver)
x = sp.linalg.spsolve(A, b, use_umfpack=True)

In [ ]:
x

In [ ]:
x, info = sp.linalg.cg(A, b)

In [ ]:
x

In [ ]:
x, info = sp.linalg.bicgstab(A, b)

In [ ]:
x

In [ ]:
# atol argument is a recent addition
x, info = sp.linalg.lgmres(A, b, atol=1e-5)

In [ ]:
x

In [ ]:
N = 25

### An example of a matrix reording method: Reverse Cuthil McKee

In [ ]:
A = sp.diags([1, -2, 1], [8, 0, -8], shape=[N, N], format='csc')

In [ ]:
perm = sp.csgraph.reverse_cuthill_mckee(A)
perm

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
ax1.spy(A)
ax2.spy(sp_permute(A, perm, perm))

### Performance comparison sparse/dense

In [ ]:
# compare performance of solving Ax=b vs system size N,
# where A is the sparse matrix for the 1d poisson problem
import time

def setup(N):
    A = sp.diags([1,-2,1], [1,0,-1], shape=[N, N], format='csr')
    b = -np.ones(N)
    return A, A.todense(), b

reps = 10
N_vec = np.arange(2, 300, 1)
t_sparse = np.empty(len(N_vec))
t_dense = np.empty(len(N_vec))
for idx, N in enumerate(N_vec):
    A, A_dense, b = setup(N)
    t = time.time()
    for r in range(reps):
        x = np.linalg.solve(A_dense, b)
    t_dense[idx] = (time.time() - t)/reps
    t = time.time()
    for r in range(reps):
        x = sp.linalg.spsolve(A, b, use_umfpack=True)
    t_sparse[idx] = (time.time() - t)/reps
    
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(N_vec, t_dense * 1e3, '.-', label="dense")
ax.plot(N_vec, t_sparse * 1e3, '.-', label="sparse")
ax.set_xlabel(r"$N$", fontsize=16)
ax.set_ylabel("elapsed time (ms)", fontsize=16)
ax.legend(loc=0)
fig.tight_layout()
fig.savefig("ch10-sparse-vs-dense.pdf")

### Eigenvalue problems

In [ ]:
N = 10

In [ ]:
A = sp.diags([1, -2, 1], [1, 0, -1], shape=[N, N], format='csc')

In [ ]:
evals, evecs = sp.linalg.eigs(A, k=4, which='LM')

In [ ]:
evals

In [ ]:
np.allclose(A.dot(evecs[:,0]), evals[0] * evecs[:,0])

In [ ]:
evals, evecs = sp.linalg.eigsh(A, k=4, which='LM')

In [ ]:
evals

In [ ]:
evals, evecs = sp.linalg.eigs(A, k=4, which='SR')

In [ ]:
evals

In [ ]:
np.real(evals).argsort()

In [ ]:
def sp_eigs_sorted(A, k=6, which='SR'):
    """ compute and return eigenvalues sorted by real value """
    evals, evecs = sp.linalg.eigs(A, k=k, which=which)
    idx = np.real(evals).argsort()
    return evals[idx], evecs[idx]

In [ ]:
evals, evecs = sp_eigs_sorted(A, k=4, which='SM')

In [ ]:
evals

#### Random matrix example

In [ ]:
N = 100

In [ ]:
x_vec = np.linspace(0, 1, 50)

In [ ]:
# seed sp.rand with random_state to obtain a reproducible result
M1 = sp.rand(N, N, density=0.2, random_state=112312321)
# M1 = M1 + M1.conj().T
M2 = sp.rand(N, N, density=0.2, random_state=984592134)
# M2 = M2 + M2.conj().T

In [ ]:
evals = np.array([sp_eigs_sorted((1-x)*M1 + x*M2, k=25)[0] for x in x_vec])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

for idx in range(evals.shape[1]):
    ax.plot(x_vec, np.real(evals[:,idx]), lw=0.5)

ax.set_xlabel(r"$x$", fontsize=16)
ax.set_ylabel(r"eig.vals. of $(1-x)M_1+xM_2$", fontsize=16)

fig.tight_layout()
fig.savefig("ch10-sparse-eigs.pdf")

## Graphs

In [ ]:
g = nx.MultiGraph()

In [ ]:
g.add_node(1)

In [ ]:
g.nodes()

In [ ]:
g.add_nodes_from([3, 4, 5])

In [ ]:
g.nodes()

In [ ]:
g.add_edge(1, 2)

In [ ]:
g.edges()

In [ ]:
g.add_edges_from([(3, 4), (5, 6)])

In [ ]:
g.edges()

In [ ]:
g.add_weighted_edges_from([(1, 3, 1.5), (3, 5, 2.5)])

In [ ]:
g.edges()

In [ ]:
g.edges(data=True)

In [ ]:
g.add_weighted_edges_from([(6, 7, 1.5)])

In [ ]:
g.nodes()

In [ ]:
g.edges()

In [ ]:
import numpy as np

In [ ]:
import json

In [ ]:
with open("tokyo-metro.json") as f:
    data = json.load(f)

In [ ]:
data.keys()

In [ ]:
data["C"]

In [ ]:
# data

In [ ]:
g = nx.Graph()

for line in data.values():
    g.add_weighted_edges_from(line["travel_times"])
    g.add_edges_from(line["transfers"])

In [ ]:
for n1, n2 in g.edges():
    g[n1][n2]["transfer"] = "weight" not in g[n1][n2]

In [ ]:
g.number_of_nodes()

In [ ]:
list(g.nodes())[:5]

In [ ]:
g.number_of_edges()

In [ ]:
list(g.edges())[:5]

In [ ]:
on_foot = [edge for edge in g.edges() if g.get_edge_data(*edge)["transfer"]]

In [ ]:
on_train = [edge for edge in g.edges() if not g.get_edge_data(*edge)["transfer"]]

In [ ]:
colors = [data[n[0].upper()]["color"] for n in g.nodes()]

In [ ]:
# from networkx.drawing.nx_agraph import graphviz_layout

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 10))

pos = nx.drawing.nx_agraph.graphviz_layout(g, prog="neato")
nx.draw(g, pos, ax=ax, node_size=200, node_color=colors)
nx.draw_networkx_labels(g, pos=pos, ax=ax, font_size=6)
nx.draw_networkx_edges(g, pos=pos, ax=ax, edgelist=on_train, width=2)
nx.draw_networkx_edges(g, pos=pos, ax=ax, edgelist=on_foot, edge_color="blue")

# removing the default axis on all sides:
for side in ['bottom','right','top','left']:
    ax.spines[side].set_visible(False)

# removing the axis labels and ticks
ax.set_xticks([])
ax.set_yticks([])
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')

fig.savefig("ch10-metro-graph.pdf")
fig.savefig("ch10-metro-graph.png")
fig.tight_layout()

In [ ]:
g.degree()

In [ ]:
d_max = max(d for (n, d) in g.degree())

In [ ]:
[(n, d) for (n, d) in g.degree() if d == d_max]

In [ ]:
p = nx.shortest_path(g, "Y24", "C19")

In [ ]:
np.array(p)

In [ ]:
np.sum([g[p[n]][p[n+1]]["weight"] for n in range(len(p)-1) if "weight" in g[p[n]][p[n+1]]])

In [ ]:
h = g.copy()

In [ ]:
for n1, n2 in h.edges():
    if "transfer" in h[n1][n2]:
        h[n1][n2]["weight"] = 5

In [ ]:
p = nx.shortest_path(h, "Y24", "C19")

In [ ]:
np.array(p)

In [ ]:
np.sum([h[p[n]][p[n+1]]["weight"] for n in range(len(p)-1)])

In [ ]:
p = nx.shortest_path(h, "Z1", "H16")

In [ ]:
np.sum([h[p[n]][p[n+1]]["weight"] for n in range(len(p)-1)])

In [ ]:
A = nx.to_scipy_sparse_matrix(g)

In [ ]:
A

In [ ]:
perm = sp.csgraph.reverse_cuthill_mckee(A)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
ax1.spy(A, markersize=2)
ax2.spy(sp_permute(A, perm, perm), markersize=2)
fig.tight_layout()
fig.savefig("ch12-rcm-graph.pdf")

## Versions

In [ ]:
%reload_ext version_information

In [ ]:
%version_information numpy, scipy, matplotlib, networkx, pygraphviz